## BERTopic

https://www.kaggle.com/code/falloutbabe/russian-invasion-of-ukraine-news-topic-modeling/notebook

In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import os
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

import torch

!pip install umap-learn
!pip install hdbscan
!pip install bertopic
!pip install sentence-transformers
!pip install emoji==1.7.0

from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=a4ce4b0d1d78a8f15eb6dad1f82de75903737bd7e7340da3331a9c5a956630fd
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=34a0a3c690435df69a59f75771b24d5ee3c8521b432f40ac56d09e94512fc9e8
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 36.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metad

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import emoji
import re


def preprocess_text(text: str) -> str:
  soup = BeautifulSoup(text, features="html.parser")
  text = soup.get_text()
  text = text.replace("\n", "")
  return text


def remove_url(text: str) -> str:
  return re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', ' ', text)

In [ ]:
news_df = pd.read_csv("post10k.csv", encoding="utf-8")
news_df = news_df.dropna(subset=['text'])
news_df = news_df.reset_index(drop=True)
news_df.text = news_df.text.apply(lambda x: preprocess_text(x))
news_df.text = news_df.text.apply(lambda x: remove_url(x))
news_df['text'] = news_df["text"].apply(lambda x: emoji.replace_emoji(x))

In [ ]:
news_df

,Unnamed: 0,msg_id,entity_id,text,content_type,keyboard,silent,has_media_spoiler,posted
0,0,5301,1002094480,Яркие фотокарточки первой победы в году уже ж...,media_group,NaN,0,0,2023-01-09 17:10:06
1,1,5302,1002094480,Кто стал обладателем игровой майки и брендиро...,text,NaN,0,0,2023-01-10 14:00:11
2,2,5304,1002094480,Тренируемся в традиционном режиме Готовимся к ...,video,NaN,0,0,2023-01-10 14:40:25
3,3,5305,1002094480,Проведи год вместе с Динамо-Ак Барс Представля...,video,NaN,0,0,2023-01-11 12:30:23
4,4,5306,1002094480,Динамо-Ак Барс МинчанкаКто окажется сильнее в...,photo,NaN,0,0,2023-01-11 16:00:06
...,...,...,...,...,...,...,...,...,...
11078,2217,5732,1150141694,Чистили дорогу-сковырнули таксофон.@montajniklvs,photo,NaN,0,0,2023-03-27 09:01:43
11079,2218,5733,1150141694,Наткнулся на полезный канал от Тимура Евгажуко...,photo,NaN,0,0,2023-03-27 09:21:34
11080,2219,5735,1150141694,@montajniklvs,media_group,NaN,0,0,2023-03-27 14:00:54
11081,2220,5736,1150141694,Компания X-Сom совместно c Hyperline приглашае...,photo,NaN,0,0,2023-03-27 14:09:43


In [6]:
docs = news_df.text.to_list()

umap_model = UMAP(n_neighbors=12, n_components=5, metric='cosine', low_memory=False)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('russian') + stopwords.words('english'))
hdbscan_model = HDBSCAN(min_cluster_size=35, min_samples=20, metric='euclidean', prediction_data=True)

topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       nr_topics=150, top_n_words=10, language='multilingual', verbose=True).fit(docs)

Batches:   0%|          | 0/347 [00:00<?, ?it/s]

2023-10-09 18:35:47,124 - BERTopic - Transformed documents to Embeddings
2023-10-09 18:36:29,301 - BERTopic - Reduced dimensionality
2023-10-09 18:36:29,961 - BERTopic - Clustered reduced embeddings
2023-10-09 18:36:32,688 - BERTopic - Reduced number of topics from 52 to 52


In [8]:
df_res = topic_model.get_topic_info()
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4757,-1_это_россии_года_rgrunews,"[это, россии, года, rgrunews, году, которые, т...",[Я: #СдаюЖильё В: #КобулетиОписание: В нашем у...
1,0,72,0_предоплату_кидала_мошенничества_просит,"[предоплату, кидала, мошенничества, просит, пр...",[​ВНИМАНИЕ - МОШЕННИКИУчастились случаи мошенн...
2,1,48,1____,"[, , , , , , , , , ]","[ , , ]"
3,2,47,2_франции_пенсионной_реформы_против,"[франции, пенсионной, реформы, против, rgrunew...",[Сенат Франции принял закон о пенсионной рефор...
4,3,103,3_минут_привет_история_статья,"[минут, привет, история, статья, материал, рас...","[Привет!Лучшее получает тот, кто умеет ждать. ..."
5,4,57,4_объявлений_тбилисиописание_хозяинконтакты_ук...,"[объявлений, тбилисиописание, хозяинконтакты, ...",[Я: #СдаюЖильё В: #ТбилисиОписание: ${ }Геолок...
6,5,667,5_объявлений_тбилисиописание_цена_сдаюжильё,"[объявлений, тбилисиописание, цена, сдаюжильё,...",[Я: #СдаюЖильё В: #ТбилисиОписание: 60 кВ м Са...
7,6,81,6_объявлений_хозяинконтакты_указанаживотные_ar...,"[объявлений, хозяинконтакты, указанаживотные, ...",[Я: #СдаюЖильё В: #БатумиОписание: Сдаётся ком...
8,7,461,7_объявлений_цена_батумиописание_сдаюжильё,"[объявлений, цена, батумиописание, сдаюжильё, ...",[Я: #СдаюЖильё В: #КобулетиОписание: Сдается к...
9,8,66,8_землетрясения_землетрясение_sputniktj_магнит...,"[землетрясения, землетрясение, sputniktj, магн...",[ В ГБАО снова землетрясение.Подземные толчки ...


In [9]:
topic_model.visualize_hierarchy()

In [10]:
topic_model.visualize_topics()

In [11]:
text = "Сдаю жилье в Грузии"
topic_model.transform(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

2023-10-09 18:38:12,036 - BERTopic - Reduced dimensionality
2023-10-09 18:38:12,047 - BERTopic - Predicted clusters


([5], array([1.]))

## keyBERT
https://github.com/MaartenGr/KeyBERT

In [7]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=cb4c9c25b82d61381b506d781426ee7a637df6b46bf1a464561268fa95f30df8
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
Successfully built keybert


In [ ]:
pip install -U sentence-transformers

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

In [ ]:
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
kw_model = KeyBERT(model=sentence_model)

In [ ]:
news_df

,Unnamed: 0,msg_id,entity_id,text,content_type,keyboard,silent,has_media_spoiler,posted
0,0,5301,1002094480,Яркие фотокарточки первой победы в году уже ж...,media_group,NaN,0,0,2023-01-09 17:10:06
1,1,5302,1002094480,Кто стал обладателем игровой майки и брендиро...,text,NaN,0,0,2023-01-10 14:00:11
2,2,5304,1002094480,Тренируемся в традиционном режиме Готовимся к ...,video,NaN,0,0,2023-01-10 14:40:25
3,3,5305,1002094480,Проведи год вместе с Динамо-Ак Барс Представля...,video,NaN,0,0,2023-01-11 12:30:23
4,4,5306,1002094480,Динамо-Ак Барс МинчанкаКто окажется сильнее в...,photo,NaN,0,0,2023-01-11 16:00:06
...,...,...,...,...,...,...,...,...,...
11078,2217,5732,1150141694,Чистили дорогу-сковырнули таксофон.@montajniklvs,photo,NaN,0,0,2023-03-27 09:01:43
11079,2218,5733,1150141694,Наткнулся на полезный канал от Тимура Евгажуко...,photo,NaN,0,0,2023-03-27 09:21:34
11080,2219,5735,1150141694,@montajniklvs,media_group,NaN,0,0,2023-03-27 14:00:54
11081,2220,5736,1150141694,Компания X-Сom совместно c Hyperline приглашае...,photo,NaN,0,0,2023-03-27 14:09:43


In [ ]:
news_df2 = news_df.copy()
news_df2 = news_df2[news_df2.text.str.len() > 200]

In [ ]:
news_df2 = news_df2.reset_index(drop=True)
news_df2

,Unnamed: 0,msg_id,entity_id,text,content_type,keyboard,silent,has_media_spoiler,posted
0,1,5302,1002094480,Кто стал обладателем игровой майки и брендиро...,text,NaN,0,0,2023-01-10 14:00:11
1,3,5305,1002094480,Проведи год вместе с Динамо-Ак Барс Представля...,video,NaN,0,0,2023-01-11 12:30:23
2,4,5306,1002094480,Динамо-Ак Барс МинчанкаКто окажется сильнее в...,photo,NaN,0,0,2023-01-11 16:00:06
3,19,5323,1002094480,Одерживаем победу сегодня! Ждём вас на нашем с...,photo,NaN,0,0,2023-01-13 20:31:24
4,25,5337,1002094480,Время брать реванш! Отправляемся в Нижний Новг...,video,NaN,0,0,2023-01-16 11:20:25
...,...,...,...,...,...,...,...,...,...
8216,2174,5642,1150141694,"ВидеонаблЯдение на автобазе.Собственно, чёрная...",media_group,NaN,0,0,2023-03-17 09:01:05
8217,2208,5711,1150141694,"Неумолимая ярость стихии, сдобренная нарушение...",media_group,NaN,0,0,2023-03-25 09:01:53
8218,2214,5727,1150141694,Межсетевое взаимодействие в сетях на базе TCP-...,web_preview,NaN,0,0,2023-03-26 20:26:30
8219,2218,5733,1150141694,Наткнулся на полезный канал от Тимура Евгажуко...,photo,NaN,0,0,2023-03-27 09:21:34


In [ ]:
kw_model.extract_keywords(news_df2.loc[38].text, keyphrase_ngram_range=(1, 1), stop_words=None)

[('россию', 0.4512),
 ('задержан', 0.4051),
 ('заключении', 0.3565),
 ('отравления', 0.2945),
 ('аэропорта', 0.2855)]

In [ ]:
news_df2.loc[38].text

'730 дней — столько находится в заключении Алексей НавальныйРовно 2 года назад, 17 января 2021 года, он вернулся в Россию из Германии, где лечился от последствий отравления. В тот же день оппозиционер был задержан в серой зоне аэропорта Шереметьево'